# Homework exercise 2
## Deadline: upload to Moodle by 3 December 18:00 h

__Suggestion: take this notebook and simply add your code and explanations. Your submission needs to include your code's output (even if the code throws an error).__

If you prefer to use .py files, you are expected to also include a PDF containing the output of your code and your explanations. Still, the code needs to be in a form that can be easily run on another computer.

If you use any file paths, assign paths to variables at the beginning of your code and use those variables when later referring to the paths.

__Name 1:__

__Name 2:__

__Name 3:__

The name of the file that you upload should be named *Homework1_YourLastName_YourStudentID*.

Reminder: you are required to attend class on 4 December to earn points for this homework exercise unless you have a valid reason for your absence.

You are encouraged to work on this exercise in teams of up to three students. If any part of the questions is unclear, please ask on the Moodle forum.

#### Options Pricing via Monte Carlo Simulations and Analytical Methods

In this problem set, you will be asked to implement analytical as well as simulation-based methods for plain vanilla options —standardized contracts with no additional features—, as well as simulation-based methods for some more complex exotic options.

You may skip the next section if you are already familiar with the topic.

#### Introduction to pricing plain vanilla options

Among the various types of options, plain vanilla European options are the most fundamental. A European option is characterized by the right, but not the obligation, to buy (in the case of a call option) or sell (in the case of a put option) an underlying asset at a predetermined strike price on a specific expiration date.

##### Payoff Functions

The payoff function for a European call option is defined as:

$\text{Payoff}_{\text{call}} = \max(S_T - K, 0)$
    
where $S_T$ is the price of the underlying asset at maturity, and $K$ is the strike price. This means that if the asset price at maturity is above the strike price, the holder can purchase the asset at the strike price and realize a gain equal to the difference. If the asset price is below the strike price, the option expires worthless, and the loss is limited to the premium paid for the option.

Conversely, the payoff function for a European put option is: 

$\text{Payoff}_{\text{put}} = \max(K - S_T, 0)$

This reflects the holder's right to sell the asset at the strike price. The put option is valuable if the asset price falls below the strike price, allowing the holder to sell the asset at a higher price than the market offers.

##### The Black-Scholes Formula

For a non-dividend-paying asset, the Black-Scholes price of a European call option is given by:

$C(S, t) = S N(d_1) - Ke^{-rt} N(d_2)$

and the price of a European put option is given by:

$P(S, t) = Ke^{-rt} N(-d_2) - S N(-d_1)$

where:

* $C(S,t)$ and $P(S,t)$ are the call and put option prices, respectively.
* $S$ is the current price of the underlying asset.
* $K$ is the strike price.
* $r$ is the risk-free interest rate.
* $t$ is the time to expiration.
* $N(⋅)$ is the cumulative distribution function of the standard normal distribution.
* $d_1=\frac{ln(S/K)+(r+\sigma^2/2)t}{\sigma\sqrt{t}}$ and $d_2=d_1-\sigma\sqrt{t}$, where $\sigma$ is the volatility of the asset's returns.


#### Options pricing via simulations

Pricing options via Monte Carlo simulations consists of

* randomly drawing stock price paths based on an assumed distribution
* computing the option payoff for each path and discounting the payoff to the present
* averaging those discounted values

It is assumed that the stock price follows a log-normal distribution, i.e., logarithmic returns follow a normal distribution. The mean return used in the simulations is the risk-free rate, the standard deviation is the assumed volatility (called "implied volatility", as this volatility is implied by the option price).

1. Implement the Black-Scholes formula for both calls and puts. Plot prices for varying strike prices, volatility, and time to maturity in three-dimensional graphs (each time holding one of these dimensions and the stock price fixed). 

2. Implement a simulator of stock prices. It needs to accept at least the following inputs:

* risk-free rate
* volatility
* current price
* unit of time for simulation (minutes, hours, days)
* time to maturity (measured in days)
* number of price paths to simulate

Use the simulator to price plain vanilla options. For parameter values of your choice, make a plot showing how the difference between prices obtained from the simulation and from the analytical formula changes with the number of simulated paths.

3. You will now turn to pricing options that cannot be priced analytically. In this task, you are asked to price Asian options. The payoff of Asian options depends on the average price of the underlying asset over a certain period.
The payoff of an Asian Call Option is

$C_{\text{Asian}}(T) = \max\left(\bar{S}(T) - K, 0\right)$

and that of an Asian Put Option

$P_{\text{Asian}}(T) = \max\left(K - \bar{S}(T), 0\right)$

where $\bar{S}(T)$ represents the average price of the underlying asset over a set of days. For simplicity, we will focus on the type of Asian option that considers the average over the last $D$ days until maturity (so-called Asian-Out options). While both arithmetic and geometric averages can be used in Asian options, we will consider only the arithmetic average here.

Determine the prices of Asian options using Monte Carlo simulations. Your implementation should allow for variation in the number of days $D$ used for the computation of $\bar{S}(T)$, which is then defined as the average over days $T-D-1$ to $T$. Make plots comparing the prices of plain vanilla and Asian options with otherwise identical parameter values.

4. We will finally consider Barrier options. Barrier options are a type of exotic option where the payoff depends on whether the underlying asset's price reaches a certain level, known as the barrier, during the option's life.

There are two main types of barrier options: knock-in and knock-out. A knock-in option becomes active only if the price of the underlying asset crosses a certain level. By contrast, a knock-out option expires worthless if the price of the underlying asset ever crosses a certain level. The options can further be distinguished by whether the price threshold is crossed from below (Up-and-out, Up-and-in) or above (Down-and-out, Down-and-in). Each of these four types of options exist for both calls and puts, thus yielding eight different payoff functions. At maturity the payoff of a barrier option, if it is active, is identical to that of an otherwise similar plain vanilla call or put option.

Determine the prices of barrier options using Monte Carlo simulations. Make plots comparing the prices of plain vanilla and Barrier options with otherwise identical parameter values, and show how the difference in prices depends on the distance of the present stock price from the barrier.